### Create Grafana Dashboard

In [1]:
import nuclio
import os

In [2]:
%run ../../set_env.ipynb

In [3]:
# nuclio: start-code

In [4]:
from grafwiz import *
import os
import v3io_frames as v3f

In [5]:
def handler(context, event):
    
    raw_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('shardId')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('RAW_VIDEO_STREAM')}"
    tagged_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('shardId')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('TAGGED_VIDEO_STREAM')}"
    
    params_js = """{
         from:ctrl.range.from.format('x'),  // x is unix ms timestamp
         to:ctrl.range.to.format('x'), 
         height:ctrl.height,
         now:Date.now(),
         interval: ctrl.template('$__interval'),
         sample: 'Not escaped: $__interval',
         since:ctrl.lastRequestTime
        }"""
    
    dash = Dashboard("Stream Viewer Test", start='now', dataSource='iguazio', refresh='1s')
    
    raw_stream = Ajax(title='Raw Stream',
                      url=raw_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    tagged_stream = Ajax(title='Tagged Stream',
                      url=tagged_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    dash.row([raw_stream, tagged_stream])
    
    return dash.deploy('http://grafana')

In [6]:
# nuclio: end-code

In [7]:
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('create-grafana-dashboard', kind='job', handler="handler", project=os.getenv("PROJECT"))

In [8]:
fn.spec.build.image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/{os.getenv('DOCKER_IMAGE')}"
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1

fn.set_env('API_GATEWAY', os.getenv('API_GATEWAY'))
fn.set_env('PROJECT', os.getenv('PROJECT'))
fn.set_env('IGZ_NAMESPACE_DOMAIN', os.getenv('IGZ_NAMESPACE_DOMAIN'))
fn.set_env('RAW_VIDEO_STREAM' ,os.getenv('RAW_VIDEO_STREAM'))
fn.set_env('TAGGED_VIDEO_STREAM', os.getenv('TAGGED_VIDEO_STREAM'))
fn.set_env('IGZ_CONTAINER', os.getenv('IGZ_CONTAINER'))
fn.set_env('shardId', os.getenv('shardId'))

fn.export("../yaml/create-grafana-dashboard.yaml")

> 2020-10-27 23:11:39,450 [info] function spec saved to path: ../yaml/create-grafana-dashboard.yaml
